# Text Summarization of Amazon reviews

In this notebook I will write summaries with the help of my Seq2Seq model in Summarizer.py.

The model works impressively well in the end!

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [1]:
import os

import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from collections import Counter

import Summarizer
import summarizer_data_utils
import summarizer_model_utils

In [3]:
print(tf.__version__)

1.14.0


## The data


The data we will be using with is a dataset from Kaggle, the Amazon Fine Food Reviews dataset.  
It contains, as the name suggests, 570.000 reviews of fine foods from Amazon and summaries of those reviews. 
Our aim is to input a review (Text column) and automatically create a summary (Summary colum) for it.


https://www.kaggle.com/snap/amazon-fine-food-reviews/data

### Reading and exploring

In [4]:
# load csv file using pandas.
#file_path = './Reviews.csv'
#data = pd.read_csv(file_path)
#data.shape

In [5]:
# we will only use the last two columns Summary (target) and Text (input).
#data.head()

In [6]:
# check for missings --> got some in summary drop those. 
# 26 are missing, so we will drop those!
#data.isnull().sum()

In [7]:
# drop row, if values in Summary is missing. 
#data.dropna(subset=['Summary'],inplace = True)

In [8]:
# only summary and text are useful for us.
#data = data[['Summary', 'Text']]
#data.head()

In [9]:
# we will not use all of them, only short ones and ones of similar size. 
# choosing the ones that are of similar length makes it easier for the model to learn.
'''
raw_texts = []
raw_summaries = []

for text, summary in zip(data.Text, data.Summary):
    if 100< len(text) < 150:
        raw_texts.append(text)
        raw_summaries.append(summary)
        '''

'\nraw_texts = []\nraw_summaries = []\n\nfor text, summary in zip(data.Text, data.Summary):\n    if 100< len(text) < 150:\n        raw_texts.append(text)\n        raw_summaries.append(summary)\n        '

In [10]:
#len(raw_texts), len(raw_summaries)

In [11]:
#for t, s in zip(raw_texts[:5], raw_summaries[:5]):
 #   print('Text:\n', t)
  #  print('Summary:\n', s, '\n\n')

### Clean and prepare the data

In [12]:
# the function gives us the option to keep_most of the characters inisde the texts and summaries, meaning
# punctuation, question marks, slashes...
# or we can set it to False, meaning we only want to keep letters and numbers like here.
'''
import nltk
nltk.download('punkt')
processed_texts, processed_summaries, words_counted = summarizer_data_utils.preprocess_texts_and_summaries(
    raw_texts,
    raw_summaries,
    keep_most=False
)
print(processed_texts[:3])
print(processed_summaries[:3])
print(len(words_counted))
print(words_counted[:5])
'''

"\nimport nltk\nnltk.download('punkt')\nprocessed_texts, processed_summaries, words_counted = summarizer_data_utils.preprocess_texts_and_summaries(\n    raw_texts,\n    raw_summaries,\n    keep_most=False\n)\nprint(processed_texts[:3])\nprint(processed_summaries[:3])\nprint(len(words_counted))\nprint(words_counted[:5])\n"

In [2]:
#for t,s in zip(processed_texts[:5], processed_summaries[:5]):
 #   print('Text\n:', t, '\n')
  #  print('Summary:\n', s, '\n\n\n')

import pandas as pd
import jieba
import tensorflow.keras as keras
max_features=10000
maxlen=300
dt=pd.read_csv("sgnewfull.csv",sep='\t',skiprows=1,names=['title','content'],nrows=10000)
#按行读取文件，返回文件的行字符串列表,读取stopwords.dat
def read_file(file_name):
    fp = open(file_name, "r", encoding="utf-8")
    content_lines = fp.readlines()
    fp.close()
    #去除行末的换行符，否则会在停用词匹配的过程中产生干扰
    for i in range(len(content_lines)):
        content_lines[i] = content_lines[i].rstrip("\n")
    return content_lines
def fenci(selist):
    stopwords = read_file("stopwords.dat")#读取停用词
    l=[]
    for i in selist:
        k=[]
        seg_list = jieba.cut(i)  # 默认是精确模式
        outstr=''
        i=0
        for word in seg_list:  #去除停顿词
            i+=1
            if word not in stopwords:  #如果去除停用词的话，把注释去掉，同时把下面三行加tab
                if word != '\t' and i<150 :  
                    k.append(word)
        l.append(k)
    return l

tokenizer = keras.preprocessing.text.Tokenizer(num_words=max_features,lower=True)
dt['title']=dt['title'].fillna("")
dt['content']=dt['content'].fillna("")
title=dt['title']
#title=[('s'+i+'e')for i in title]

content=fenci(list(dt['content']))
title=fenci(title)
#统计词
words=[]
for text in content:
    for word in text:
        words.append(word)
for text in title:
    for word in text:
        words.append(word)
words_counted = Counter(words).most_common() 
print(words_counted[:3])
print(content[:3])     
print(title[:3])
'''
processed_content, processed_title, words_counted = summarizer_data_utils.preprocess_texts_and_summaries(
    content,
    title,
    keep_most=False
)'''



#tokenizer.fit_on_texts(content + 
 #                      title)
#content=tokenizer.texts_to_sequences(content)
#content=keras.preprocessing.sequence.pad_sequences(content,maxlen=maxlen) # padding
#title=tokenizer.texts_to_sequences(title)



Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.632 seconds.
Prefix dict has been built successfully.
[('的', 43987), ('１', 20805), ('０', 19880)]
[['南', '都', '讯', '\u3000', '记者', '刘凡', '\u3000', '周昌', '和', '\u3000', '任笑', '一', '\u3000', '继', '推出', '日票', '后', '深圳', '今后', '将', '设', '地铁', 'Ｖ', 'Ｉ', 'Ｐ', '头等', '车厢', '设', '坐票', '制', '昨日', '《', '南', '都', 'Ｍ', 'Ｅ', 'Ｔ', 'Ｒ', 'Ｏ', '》', '创刊', '仪式', '暨', '２', '０', '１', '２', '年', '深港', '地铁', '圈', '高峰论坛', '上', '透露', '在', '未来', '的', '１', '１', '号线', '上将', '增加', '特色', '服务', '满足', '不同', '消费', '层次', '的', '乘客', '的', '不同', '需求', '如', '特设', '行李架', '的', '车厢', '和', '买', '双倍', '票', '可', '有', '座位', '坐', '的', 'Ｖ', 'Ｉ', 'Ｐ', '车厢', '等', '\ue40c', '论坛', '上', '深圳市政府', '副', '秘书长', '轨道交通', '建设', '办公室', '主任', '赵鹏林', '透露', '地铁', '未来', '的', '方向', '将', '分等级', '满足', '不同', '层次', '的', '人', '的', '需求', '提供', '不同', '层次', '的', '有', '针对', '的', '服务', '其中', '包括', '一些', '档次', '稍微', '高', '一些', '的', '服务'], ['同心县', '地

'\nprocessed_content, processed_title, words_counted = summarizer_data_utils.preprocess_texts_and_summaries(\n    content,\n    title,\n    keep_most=False\n)'

### Create lookup dicts

We cannot feed our network actual words, but numbers. So we first have to create our lookup dicts, where each words gets and int value (high or low, depending on its frequency in our corpus). Those help us to later convert the texts into numbers.

We also add special tokens. EndOfSentence and StartOfSentence are crucial for the Seq2Seq model we later use.
Pad token, because all summaries and texts in a batch need to have the same length, pad token helps us do that.

So we need 2 lookup dicts:
 - From word to index 
 - from index to word. 

In [3]:
specials = ["<EOS>", "<SOS>","<PAD>","<UNK>"]
word2ind, ind2word,  missing_words = summarizer_data_utils.create_word_inds_dicts(words_counted,
                                                                       specials = specials)
print(len(word2ind), len(ind2word), len(missing_words))


80304 80304 0


### Pretrained embeddings

Optionally we can use pretrained word embeddings. Those have proved to increase training speed and accuracy.
Here I used two different options. Either we use glove embeddings or embeddings from tf_hub.
The ones from tf_hub worked better, so we use those. 

In [15]:
# glove_embeddings_path = './glove.6B.300d.txt'
# embedding_matrix_save_path = './embeddings/my_embedding_github.npy'
# emb = summarizer_data_utils.create_and_save_embedding_matrix(word2ind,
#                                                              glove_embeddings_path,
#                                                              embedding_matrix_save_path)

In [16]:
# the embeddings from tf_hub. 
# embed = hub.Module("https://tfhub.dev/google/nnlm-en-dim128/1")
'''
embed = hub.Module("https://tfhub.dev/google/Wiki-words-250/1")
emb = embed([key for key in word2ind.keys()])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    embedding = sess.run(emb)
'''

'\nembed = hub.Module("https://tfhub.dev/google/Wiki-words-250/1")\nemb = embed([key for key in word2ind.keys()])\n\nwith tf.Session() as sess:\n    sess.run(tf.global_variables_initializer())\n    sess.run(tf.tables_initializer())\n    embedding = sess.run(emb)\n'

In [17]:
#embedding.shape

In [18]:
#np.save('./tf_hub_embedding.npy', embedding)

### Convert text and summaries

As I said before we cannot feed the words directly to our network, we have to convert them to numbers first of all. This is what we do here. And we also append the SOS and EOS tokens.

In [4]:
# converts words in texts and summaries to indices
# it looks like we have to set eos here to False
processed_texts=content
converted_texts, unknown_words_in_texts = summarizer_data_utils.convert_to_inds(processed_texts,
                                                                                word2ind,
                                                                                eos = False)

In [5]:
processed_summaries=title
converted_summaries, unknown_words_in_summaries = summarizer_data_utils.convert_to_inds(processed_summaries,word2ind, eos = True,  sos = True)

In [21]:
#print(converted_texts[:3])
#print(converted_summaries[:3])

In [22]:
# seems to have worked well. 
#print( summarizer_data_utils.convert_inds_to_text(converted_texts[0], ind2word),
     #  summarizer_data_utils.convert_inds_to_text(converted_summaries[0], ind2word))


## The model

Now we can build and train our model. First we define the hyperparameters we want to use. Then we create our Summarizer and call the function .build_graph(), which as the name suggests, builds the computation graph. 
Then we can train the model using .train()

After training we can try our model using .infer()

### Training

We can optionally use a cyclic learning rate, which we do here. 
I trained the model for 20 epochs and the loss was low then, but we could train it longer and would probably get better results.

Unfortunately I do not have the resources to find the perfect (or right) hyperparameters, but these do pretty well. 


In [6]:
# model hyperparametes
num_layers_encoder = 2
num_layers_decoder = 2
rnn_size_encoder = 128
rnn_size_decoder = 128

batch_size = 64
epochs = 10
clip = 5
keep_probability = 0.5
learning_rate = 0.001
max_lr=0.005
learning_rate_decay_steps = 700
learning_rate_decay = 0.90


pretrained_embeddings_path = './tf_hub_embedding.npy'
summary_dir = os.path.join('./tensorboard', str('Nn_' + str(rnn_size_encoder) + '_Lr_' + str(learning_rate)))


use_cyclic_lr = True
inference_targets=True


In [24]:
#len(converted_summaries)

In [25]:
d=round(len(converted_summaries)*0.9)

In [26]:
#from numba import cuda
#cuda.select_device(0)
#cuda.close()

In [27]:
# build graph and train the model 
summarizer_model_utils.reset_graph()
summarizer = Summarizer.Summarizer(word2ind,
                                   ind2word,
                                   save_path='./models/sogou/my_model',
                                   mode='TRAIN',
                                   num_layers_encoder = num_layers_encoder,
                                   num_layers_decoder = num_layers_decoder,
                                   rnn_size_encoder = rnn_size_encoder,
                                   rnn_size_decoder = rnn_size_decoder,
                                   batch_size = 32,
                                   clip = clip,
                                   keep_probability = keep_probability,
                                   learning_rate = learning_rate,
                                   max_lr=max_lr,
                                   learning_rate_decay_steps = learning_rate_decay_steps,
                                   learning_rate_decay = learning_rate_decay,
                                   epochs = epochs,
                                   pretrained_embeddings_path = None, #pretrained_embeddings_path,
                                   use_cyclic_lr = use_cyclic_lr,
                                   summary_dir = None)#summary_dir)           

summarizer.build_graph()
summarizer.train(converted_texts[:d], 
                 converted_summaries[:d],
                 validation_inputs=converted_texts[d:],
                 validation_targets=converted_summaries[d:])


# hidden training output.
# both train and validation loss decrease nicely.





Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-cont

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[25,64,722908] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node dynamic_seq2seq/decoder/decoder/TensorArrayStack/TensorArrayGatherV3 (defined at /root/my/Summarizer.py:302) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[dynamic_seq2seq/Adam/update/_182]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[25,64,722908] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node dynamic_seq2seq/decoder/decoder/TensorArrayStack/TensorArrayGatherV3 (defined at /root/my/Summarizer.py:302) ]]
Hint: If you want to see a list of a

### Inference
Now we can use our trained model to create summaries. 

In [7]:
summarizer_model_utils.reset_graph()
summarizer = Summarizer.Summarizer(word2ind,
                                   ind2word,
                                   './models/sogou/my_model',
                                   'INFER',
                                   num_layers_encoder = num_layers_encoder,
                                   num_layers_decoder = num_layers_decoder,
                                   batch_size = len(converted_texts[:50]),
                                   clip = clip,
                                   keep_probability = 1.0,
                                   learning_rate = 0.0,
                                   beam_width = 5,
                                   rnn_size_encoder = rnn_size_encoder,
                                   rnn_size_decoder = rnn_size_decoder,
                                   inference_targets = True,
                                   pretrained_embeddings_path = None)#pretrained_embeddings_path)

summarizer.build_graph()
preds = summarizer.infer(converted_texts[:50],
                         restore_path =  './models/sogou/my_model',
                         targets = converted_summaries[:50])






Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on funct

In [8]:
# show results
summarizer_model_utils.sample_results(preds,
                                      ind2word,
                                      word2ind,
                                      converted_summaries[:50],
                                      converted_texts[:50])




 ----------------------------------------------------------------------------------------------------
Actual Text:
南 都 讯 　 记者 刘凡 　 周昌 和 　 任笑 一 　 继 推出 日票 后 深圳 今后 将 设 地铁 Ｖ Ｉ Ｐ 头等 车厢 设 坐票 制 昨日 《 南 都 Ｍ Ｅ Ｔ Ｒ Ｏ 》 创刊 仪式 暨 ２ ０ １ ２ 年 深港 地铁 圈 高峰论坛 上 透露 在 未来 的 １ １ 号线 上将 增加 特色 服务 满足 不同 消费 层次 的 乘客 的 不同 需求 如 特设 行李架 的 车厢 和 买 双倍 票 可 有 座位 坐 的 Ｖ Ｉ Ｐ 车厢 等  论坛 上 深圳市政府 副 秘书长 轨道交通 建设 办公室 主任 赵鹏林 透露 地铁 未来 的 方向 将 分等级 满足 不同 层次 的 人 的 需求 提供 不同 层次 的 有 针对 的 服务 其中 包括 一些 档次 稍微 高 一些 的 服务

Actual Summary:
深圳 地铁 将 设立 Ｖ Ｉ Ｐ 头等 车厢 　 买 双倍 票 可享 坐票

Created Summary:
中国 中国 中国 中国 　 　 　 称 称





 ----------------------------------------------------------------------------------------------------
Actual Text:
同心县 地处 宁夏 中部 干旱 带 的 核心区 　 冬寒 长 春暖迟 夏热 短 秋凉 早 干旱 少雨 蒸发 强烈 风大沙 多 主要 自然灾害 有 沙尘暴 干热风 霜冻 冰雹 等 其中 以 干旱 危害 最为 严重  由于 生态环境 的 极度 恶劣 导致 农村 经济 发展缓慢 人民 群众 生产 生活 水平 低下 靠天吃饭 的 被动局面 依然 存在 同心 又 是 国家级 老 少 边 穷县 之一 … ［ 详细 ］

Actual Summary:
中国 西部 是 地球 上 主要 干旱 带 之一 妇女 是 当地 劳动力 ． ． ．

Created Summary:
２ ２ ２ ２ ０ ０ ０ ０ ０ ０ ０ ０ ０ ０





 ---

# Conclusion

Generally I am really impressed by how well the model works. 
We only used a limited amount of data, trained it for a limited amount of time and used nearly random hyperparameters and it still delivers good results. 

However, we are clearly overfitting the training data and the model does not perfectly generalize.
Sometimes the summaries the model creates are good, sometimes bad, sometimes they are better than the original ones and sometimes they are just really funny.


Therefore it would be really interesting to scale it up and see how it performs. 

To sum up, I am impressed by seq2seq models, they perform great on many different tasks and I look foward to exploring more possible applications. 
(speech recognition...)